## Importar librerías

In [1]:
import os
import glob
import pandas as pd
import numpy as np
from genetic_algorithm import GeneticAlgorithmCV
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import KernelPCA, TruncatedSVD
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
import xgboost as xgb
from concurrent.futures import ThreadPoolExecutor, wait
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn, update_registered_converter
from skl2onnx.common.shape_calculator import calculate_linear_classifier_output_shapes
from onnxmltools.convert.xgboost.operator_converters.XGBoost import convert_xgboost
import warnings
warnings.filterwarnings('ignore')

## Cargar y preparar datos

In [2]:
# Cargar el archivo CSV
file_folder = r"/mnt/c/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/Common/Files/"
file_pattern = os.path.join(file_folder, 'training_dataset_*.csv')
df_file_path = glob.glob(file_pattern)
df = pd.read_csv(df_file_path[0])

In [3]:
def create_training_dataset(df, trade_type):
    df = df.drop_duplicates()
    # Filtrar las operaciones del tipo especificado y con profit != 0
    df_trade = df[(df['type'] == trade_type) & (df['profit'] != 0)].copy()
    # Añadir la columna 'Target' basada en el profit
    df_trade['target'] = df_trade['profit'].apply(lambda x: 1 if x > 0 else 0)
    # Seleccionar las columnas necesarias (todas menos las dos últimas para el conjunto principal)
    feature_columns = df.columns[:-2]
    df_trade = df_trade[feature_columns.tolist() + ['target']]
    # Eliminar posibles missings
    if(df_trade.isna().values.any()):
        df_trade = df_trade.dropna()
    # Debug info
    n_winning = len(df_trade[df_trade['target']==1])
    n_losing = len(df_trade[df_trade['target']==0])
    print(f"Tipo de operación: {'Buy' if trade_type == 1 else 'Sell'}")
    print(f"Número de características: {df.iloc[:, :-2].shape[1]}")
    print(f"Total Ganadoras: {n_winning}")
    print(f"Total Perdedoras: {n_losing}")
    print("___________________________________")
    # Preparación de los datos de entrenamiento
    X_train = df_trade.drop(columns='target').values
    y_train = df_trade['target'].values
    # return dataframe
    return X_train, y_train

# Obtener datasets de entrenamiento
X_buy, y_buy = create_training_dataset(df, 1)
X_sell, y_sell = create_training_dataset(df, -1)

Tipo de operación: Buy
Número de características: 566
Total Ganadoras: 2688
Total Perdedoras: 2291
___________________________________
Tipo de operación: Sell
Número de características: 566
Total Ganadoras: 2516
Total Perdedoras: 2009
___________________________________


## Entrenamiento de los modelos

In [4]:
# Definir algoritmo de validación cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# Definir escalador
scaler = StandardScaler()
# Definir reductor de dimensionalidad
reducer = KernelPCA(kernel='rbf')
# Definir el clasificador
classifier = xgb.XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    device = "cuda",
    verbosity=0
    )
# Definir Pipeline
pipeline = Pipeline([
        ('scaler', scaler),
        ('reducer', reducer),
        ('xgbclassifier', classifier)
        ])
# Definir espacio de hiperparámetros para compras
n_features = df.iloc[:, :-2].shape[1]
param_grid = {
    'scaler': {'type': 'categorical', 'values': [StandardScaler(), RobustScaler()]},
    'reducer': {'type': 'categorical', 'values': [KernelPCA('rbf'), KernelPCA('linear'), TruncatedSVD()]},
    'reducer__n_components': {'type': 'int', 'low': 1, 'high': n_features},
    #'reducer__kernel': {'type': 'categorical', 'values': ['linear', 'poly', 'rbf', 'sigmoid', 'cosine']},
    'xgbclassifier__n_estimators': {'type': 'int', 'low': 50, 'high': 500},
    'xgbclassifier__max_depth': {'type': 'int', 'low': 3, 'high': 10},
    'xgbclassifier__eta':{'type': 'float', 'low': 1e-3, 'high': 5e-1},
    'xgbclassifier__gamma':{'type': 'float', 'low': 0.0, 'high': 0.5},
    'xgbclassifier__subsample': {'type': 'float', 'low': 0.5, 'high': 1.0},
    'xgbclassifier__colsample_bytree': {'type': 'float', 'low': 0.5, 'high': 1.0},
    'xgbclassifier__colsample_bylevel': {'type': 'float', 'low': 0.5, 'high': 1.0},
    'xgbclassifier__colsample_bynode': {'type': 'float', 'low': 0.5, 'high': 1.0},
    'xgbclassifier__alpha': {'type': 'float', 'low': 0.0, 'high': 1.0},
    'xgbclassifier__lambda': {'type': 'float', 'low': 0.0, 'high': 1.0},
    'xgbclassifier__min_child_weight': {'type': 'int', 'low': 1, 'high': 10}
}

In [5]:
# Train model functions
def train_classifier(X_train, y_train, model_type, pipeline, cv, param_grid):
    try:
        # Entrenar el modelo utilizando el algoritmo genético
        ga_search = GeneticAlgorithmCV(
            model_type=model_type,
            pipeline=pipeline,
            param_grid=param_grid,
            cv=cv,
            pop_size=25,
            generations=15,
            early_stopping_rounds=3,
            crossover_initial=0.9,
            crossover_end=0.1,
            mutation_initial=0.1,
            mutation_end=0.9,
            elitism=True,
            elite_size=3,
            tournament_size=3,
            n_random=0,
            n_jobs=-1,
            verbose=True,
        )
        ga_search.fit(X_train, y_train)
    except Exception as e:
        print(f"Error en traing model {model_type}: {e}")
        raise
    # Obtener los mejores parámetros y el mejor estimador
    print("####################################################################")
    print(f"Mejor puntuación de validación para {model_type}: {ga_search.best_score_}")
    print(f"Mejores parámetros encontrados para {model_type}: {ga_search.best_params_}")
    print("####################################################################")
    # Retornar mejor estimador
    return ga_search.best_estimator_

# Entrenar modelos en paralelo
with ThreadPoolExecutor(max_workers=2) as executor:
    print("Esperando que las tareas finalicen...")
    # enviar tareas de entrenamiento
    future_buy = executor.submit(train_classifier, X_buy, y_buy, "buy", pipeline, cv, param_grid)
    future_sell = executor.submit(train_classifier, X_sell, y_sell, "sell", pipeline, cv, param_grid)
    # esperar a que todas las tareas terminen
    futures = [future_buy, future_sell]
    wait(futures)
    # Obtener resultados una vez que ambas tareas han terminado
    model_buy = future_buy.result()
    model_sell = future_sell.result()
    print("¡Todas las tareas han terminado!")

Esperando que las tareas finalicen...


Generaciones sell:   0%|          | 0/15 [00:00<?, ?gen/s]

Generaciones buy:   0%|          | 0/15 [00:00<?, ?gen/s]

[1, buy] Fitness: 0.6268222123771937 | Best Fitness: 0.6268222123771937
[1, buy] Fitness Improvement: 0.0000 | Diversity: 1.5274
[1, buy] Normalized Fitness Improvement: 0.0000 | Normalized Diversity: 0.0000
[1, buy] Crossover Rate: 0.9000 | Mutation Rate: 0.1000
[1, sell] Fitness: 0.6183915778533743 | Best Fitness: 0.6183915778533743
[1, sell] Fitness Improvement: 0.0000 | Diversity: 1.5132
[1, sell] Normalized Fitness Improvement: 0.0000 | Normalized Diversity: 0.0000
[1, sell] Crossover Rate: 0.9000 | Mutation Rate: 0.1000
[2, buy] Fitness: 0.6090160326273045 | Best Fitness: 0.6268222123771937
[2, buy] Fitness Improvement: -0.0178 | Diversity: 1.4577
[2, buy] Normalized Fitness Improvement: 0.0000 | Normalized Diversity: 0.0000
[2, buy] Crossover Rate: 0.7774 | Mutation Rate: 0.1158
[2, sell] Fitness: 0.610995404345387 | Best Fitness: 0.6183915778533743
[2, sell] Fitness Improvement: -0.0074 | Diversity: 1.5036
[2, sell] Normalized Fitness Improvement: 0.0000 | Normalized Diversity:

## Exportar modelos a formato ONNX

In [6]:
def save_onnx_models(mql5_files_folder, model_buy, model_sell, X_buy, X_sell):
    try:
        update_registered_converter(
            xgb.XGBClassifier,
            "XGBClassifier",
            calculate_linear_classifier_output_shapes,
            convert_xgboost,
            options={'nocl': [True, False], 'zipmap': [True, False, 'columns']}
        )
        model_buy_onnx = convert_sklearn(
            model_buy,
            'pipeline_buy_xgboost',
            [('input', FloatTensorType([None, X_buy.shape[1]]))],
            target_opset={'': 12, 'ai.onnx.ml': 2}
        )
        model_sell_onnx = convert_sklearn(
            model_sell,
            'pipeline_sell_xgboost',
            [('input', FloatTensorType([None, X_sell.shape[1]]))],
            target_opset={'': 12, 'ai.onnx.ml': 2}
        )
        with open(os.path.join(mql5_files_folder, "model_buy.onnx"), 'wb') as f:
            f.write(model_buy_onnx.SerializeToString())
        with open(os.path.join(mql5_files_folder, "model_sell.onnx"), 'wb') as f:
            f.write(model_sell_onnx.SerializeToString())
    except Exception as e:
        print(f"Error en exportar los modelos: {e}")
        raise
    print("Modelos ONNX exportados correctamente")

mql5_files_folder = r'/mnt/c/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/6C3C6A11D1C3791DD4DBF45421BF8028/MQL5/Files'
save_onnx_models(mql5_files_folder, model_buy, model_sell, X_buy, X_sell)

Modelos ONNX exportados correctamente
